In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder

from lightgbm import LGBMRegressor

from tqdm import tqdm_notebook as tqdm
from math import log
import time
import os
print(os.listdir("../input"))

['test.csv', 'mulliken_charges.csv', 'dipole_moments.csv', 'train.csv', 'structures.csv', 'magnetic_shielding_tensors.csv', 'potential_energy.csv', 'sample_submission.csv', 'scalar_coupling_contributions.csv', 'structures']


In [2]:
train_df = pd.read_csv('../input/train.csv')
structures = pd.read_csv('../input/structures.csv')

In [3]:
train_df.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074


In [4]:
train_df.shape

(4658147, 6)

### Structures

In [5]:
structures.shape

(2358657, 6)

In [6]:
structures.head()

,molecule_name,atom_index,atom,x,y,z
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397


In [7]:
def map_atom_info(df, atom_idx):
    """
    function for merging features from structures file with train!
    """
    df = pd.merge(df, structures, how = 'left',
                  left_on  = ['molecule_name', f'atom_index_{atom_idx}'],
                  right_on = ['molecule_name',  'atom_index'])
    
    df = df.drop('atom_index', axis=1)
    df = df.rename(columns={'atom': f'atom_{atom_idx}',
                            'x': f'x_{atom_idx}',
                            'y': f'y_{atom_idx}',
                            'z': f'z_{atom_idx}'})
    return df

In [8]:
train_0 = map_atom_info(train_df, 0) # atom_index_0
train_1 = map_atom_info(train_df, 1) # atom_index_1

In [9]:
train_p_0 = train_0[['x_0', 'y_0', 'z_0']].values
train_p_1 = train_1[['x_1', 'y_1', 'z_1']].values

### Feature engineering

In [10]:
# features already in raw df files
ai0 = train_df['atom_index_0']
ai1 = train_df['atom_index_1']
type_ = pd.get_dummies(train_df['type'])

# euclidean distance between two atoms feature
euclid_distance = np.linalg.norm(train_p_0 - train_p_1, axis=1) 
distance = pd.DataFrame(euclid_distance, columns=['distance'])
# which atom has larger index feature and how much far apart
larger_atom = pd.DataFrame(ai0 > ai1, columns=['larger_atom'])
index_diff = pd.DataFrame(abs(ai0 - ai1), columns=['index_diff'])

# coordinates faetures
x_0 = train_0['x_0']
y_0 = train_0['y_0']
z_0 = train_0['z_0']

x_1 = train_1['x_1']
y_1 = train_1['y_1']
z_1 = train_1['z_1']

# distance between axis feature
x_dist = pd.DataFrame(abs(x_0 - x_1), columns=['x_dist'])
y_dist = pd.DataFrame(abs(y_0 - y_1), columns=['y_dist'])
z_dist = pd.DataFrame(abs(z_0 - z_1), columns=['z_dist'])



In [11]:
final_train = pd.concat([train_df['id'],train_df['molecule_name'],train_df['type'],type_, ai0, ai1, larger_atom,index_diff,distance,x_0,y_0,z_0,x_1,y_1,z_1], axis = 1) 

In [12]:
final_train.head()

,id,molecule_name,type,1JHC,1JHN,2JHC,2JHH,2JHN,3JHC,3JHH,3JHN,atom_index_0,atom_index_1,larger_atom,index_diff,distance,x_0,y_0,z_0,x_1,y_1,z_1
0,0,dsgdb9nsd_000001,1JHC,1,0,0,0,0,0,0,0,1,0,True,1,1.091953,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001
1,1,dsgdb9nsd_000001,2JHH,0,0,0,1,0,0,0,0,1,2,False,1,1.783120,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277
2,2,dsgdb9nsd_000001,2JHH,0,0,0,1,0,0,0,0,1,3,False,2,1.783147,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644
3,3,dsgdb9nsd_000001,2JHH,0,0,0,1,0,0,0,0,1,4,False,3,1.783157,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397
4,4,dsgdb9nsd_000001,1JHC,1,0,0,0,0,0,0,0,2,0,True,2,1.091952,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001


In [13]:
def add_features(df):
    """
    add more more features
    min, max, mean etc. of special subclasses
    """
    
    df[f'molecule_atom_index_0_x_0_std'] = df.groupby(['molecule_name', 'atom_index_0'])['x_0'].transform('std')
    df[f'molecule_atom_index_0_y_0_std'] = df.groupby(['molecule_name', 'atom_index_0'])['y_0'].transform('std')
    df[f'molecule_atom_index_0_z_0_std'] = df.groupby(['molecule_name', 'atom_index_0'])['z_0'].transform('std')
    df[f'molecule_atom_index_1_x_1_std'] = df.groupby(['molecule_name', 'atom_index_1'])['x_1'].transform('std')
    df[f'molecule_atom_index_1_y_1_std'] = df.groupby(['molecule_name', 'atom_index_1'])['y_1'].transform('std')
    df[f'molecule_atom_index_1_z_1_std'] = df.groupby(['molecule_name', 'atom_index_1'])['z_1'].transform('std')
    
    df['molecule_atom_index_0_x_0_std'] = df['molecule_atom_index_0_x_0_std'].fillna(df['molecule_atom_index_0_x_0_std'].median())
    df['molecule_atom_index_0_y_0_std'] = df['molecule_atom_index_0_y_0_std'].fillna(df['molecule_atom_index_0_y_0_std'].median())
    df['molecule_atom_index_0_z_0_std'] = df['molecule_atom_index_0_z_0_std'].fillna(df['molecule_atom_index_0_z_0_std'].median())
    df['molecule_atom_index_1_x_1_std'] = df['molecule_atom_index_1_x_1_std'].fillna(df['molecule_atom_index_1_x_1_std'].median())
    df['molecule_atom_index_1_y_1_std'] = df['molecule_atom_index_1_y_1_std'].fillna(df['molecule_atom_index_1_y_1_std'].median())
    df['molecule_atom_index_1_z_1_std'] = df['molecule_atom_index_1_z_1_std'].fillna(df['molecule_atom_index_1_z_1_std'].median())
    
        
    df['molecule_dist_mean'] = df.groupby('molecule_name')['distance'].transform('mean')
    df['molecule_dist_min'] = df.groupby('molecule_name')['distance'].transform('min')
    df['molecule_dist_max'] = df.groupby('molecule_name')['distance'].transform('max')
    df['molecule_couples'] = df.groupby('molecule_name')['id'].transform('count')

    df[f'min0'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('min')
    df[f'min1'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('min')
    df[f'max0'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('max')
    df[f'max1'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('max')
    df[f'mean0'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('mean')
    df[f'mean1'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('mean')
    df['atom_0_couples_count'] = df.groupby(['molecule_name', 'atom_index_0'])['id'].transform('count')
    df['atom_1_couples_count'] = df.groupby(['molecule_name', 'atom_index_1'])['id'].transform('count')
    df[f'std0'] = df.groupby(['molecule_name', 'atom_index_0'])['distance'].transform('std')
    df[f'std1'] = df.groupby(['molecule_name', 'atom_index_1'])['distance'].transform('std')
    df[['std0','std1']] = df[['std0','std1']].fillna(df[['std0','std1']].median()) #there are Na values in std columns
    
    df[f'molecule_type_dist_max'] = df.groupby(['molecule_name', 'type'])['distance'].transform('max')
    df[f'molecule_type_dist_min'] = df.groupby(['molecule_name', 'type'])['distance'].transform('min')



In [14]:
add_features(final_train)

In [15]:
final_train = final_train.drop(['id','molecule_name','type'], axis = 1) # final dependent variables

scc = train_df['scalar_coupling_constant'] # dependent variable

In [16]:
final_train.head()

,1JHC,1JHN,2JHC,2JHH,2JHN,3JHC,3JHH,3JHN,atom_index_0,atom_index_1,larger_atom,index_diff,distance,x_0,y_0,z_0,x_1,y_1,z_1,molecule_atom_index_0_x_0_std,molecule_atom_index_0_y_0_std,molecule_atom_index_0_z_0_std,molecule_atom_index_1_x_1_std,molecule_atom_index_1_y_1_std,molecule_atom_index_1_z_1_std,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_couples,min0,min1,max0,max1,mean0,mean1,atom_0_couples_count,atom_1_couples_count,std0,std1,molecule_type_dist_max,molecule_type_dist_min
0,1,0,0,0,0,0,0,0,1,0,True,1,1.091953,0.002150,-0.006031,0.001976,-0.012698,1.085804,0.008001,0.0,0.0,0.0,0.0,0.0,0.0,1.506668,1.091946,1.783158,10,1.091953,1.091946,1.783157,1.091953,1.610344,1.091950,4,4,0.345594,0.000003,1.091953,1.091946
1,0,0,0,1,0,0,0,0,1,2,False,1,1.783120,0.002150,-0.006031,0.001976,1.011731,1.463751,0.000277,0.0,0.0,0.0,0.0,0.0,0.0,1.506668,1.091946,1.783158,10,1.091953,1.783120,1.783157,1.783120,1.610344,1.783120,4,1,0.345594,0.710133,1.783158,1.783120
2,0,0,0,1,0,0,0,0,1,3,False,2,1.783147,0.002150,-0.006031,0.001976,-0.540815,1.447527,-0.876644,0.0,0.0,0.0,0.0,0.0,0.0,1.506668,1.091946,1.783158,10,1.091953,1.783147,1.783157,1.783158,1.610344,1.783153,4,2,0.345594,0.000007,1.783158,1.783120
3,0,0,0,1,0,0,0,0,1,4,False,3,1.783157,0.002150,-0.006031,0.001976,-0.523814,1.437933,0.906397,0.0,0.0,0.0,0.0,0.0,0.0,1.506668,1.091946,1.783158,10,1.091953,1.783148,1.783157,1.783157,1.610344,1.783151,4,3,0.345594,0.000005,1.783158,1.783120
4,1,0,0,0,0,0,0,0,2,0,True,2,1.091952,1.011731,1.463751,0.000277,-0.012698,1.085804,0.008001,0.0,0.0,0.0,0.0,0.0,0.0,1.506668,1.091946,1.783158,10,1.091952,1.091946,1.783158,1.091953,1.552753,1.091950,3,4,0.399065,0.000003,1.091953,1.091946


In [17]:
# USE WHEN VALIDATING - 

# final_train.iloc[:4000000,:])
# scc.iloc[:4000000]

# final_train.iloc[4000000:,:])
# scc.iloc[4000000:])

In [18]:
def compute_score(df,y_actual):
        """
        function for calculating predefined score
        first 8 columns of df needs to be one-hot encoded "type" feature!
        """
        score = 0
        
        y_predicted = model_rf.predict(df)
        
        for t in range(8):
            n_t = sum(df.iloc[:,t])
            y_p = y_predicted[df.iloc[:,t] == 1]
            y_a = y_actual[df.iloc[:,t] == 1]
            type_score = log(sum(abs(y_a - y_p)) / n_t)
            print(final_train.columns[t+1] + " type sub-score: " + str(type_score))
            score = score + type_score
        
        print ("overall score: " + str(score/8))
        return score / 8

### Model training

### Random forest

In [19]:
model_rf = RandomForestRegressor(n_estimators=15,min_samples_leaf=3,verbose=500)

In [20]:
model_rf.fit(final_train,scc)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 15
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.4min remaining:    0.0s
building tree 2 of 15
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  8.7min remaining:    0.0s
building tree 3 of 15
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 13.1min remaining:    0.0s
building tree 4 of 15
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 17.5min remaining:    0.0s
building tree 5 of 15
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 21.9min remaining:    0.0s
building tree 6 of 15
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed: 26.2min remaining:    0.0s
building tree 7 of 15
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed: 30.5min remaining:    0.0s
building tree 8 of 15
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed: 34.9min remaining:    0.0s
building tree 9 of 15
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed: 39.2min remaining:    0.0s
b

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=15,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=500, warm_start=False)

### Finding optimal number of estimators

In [21]:
# preds = np.stack([e.predict(X_val) for e in model_rf.estimators_])

In [22]:
# np.mean(preds[:,0]), y_val[0]

In [23]:
# plt.plot([metrics.mean_absolute_error(y_val,np.mean(preds[:i+1],axis=0)) for i in range(10)]);
# so 10-15 estimators is okay

In [24]:
# "real" score with 10 estimators (= -0.26)

# score = compute_score(X_val,y_val)
# score

### Finding optimal max features

In [25]:
# criterion / val-score
# default / -0.26 THE BEST
# 0.5 / -0.2326
# log2 / 0.0239
# sqrt / -0.04688

In [26]:
# model_rf = RandomForestRegressor(n_estimators=10,max_features='sqrt',verbose=500)

In [27]:
# model_rf.fit(X_train,y_train)

### finding optimal min_samples_leaf

In [28]:
# min_samples_leaf / val-score
# default / -0.26
# 3 / -0.262
# 10 / -0.19166

In [29]:
# model_rf = RandomForestRegressor(n_estimators=10,min_samples_leaf=10,verbose=500)

In [30]:
# model_rf.fit(X_train,y_train)

### Max depth

In [31]:
# 5 -> not good
# 10 -> not good

# model_rf = RandomForestRegressor(n_estimators=10,min_samples_leaf=3,verbose=500)

In [32]:
# model_rf.fit(X_train,y_train)

### LGBM

In [33]:
# model_lgbm = LGBMRegressor(boosting_type="gbdt",learning_rate=0.075,num_leaves=511,verbose = 500)

In [34]:
# model_lgbm.fit(X, y)

### MODEL EVALUATION

### RF evaluation

In [35]:
score = compute_score(final_train,scc)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    6.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:   10.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   13.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   16.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   19.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   23.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   26.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   29.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   32.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:   35.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

### LGBM evaluation - turns out rf performs better

In [36]:
# X = X_train[:,:8]
# y_predicted = model_lgbm.predict(X_train)

In [37]:
# score = compute_score(X,y_predicted,y_train)

In [38]:
# score

### Making predictions on test set

In [39]:
test_df = pd.read_csv('../input/test.csv')

### preprocessing the test data

In [40]:
test_0 = map_atom_info(test_df, 0)
test_1 = map_atom_info(test_df, 1)

In [41]:
test_p_0 = test_0[['x_0', 'y_0', 'z_0']].values
test_p_1 = test_1[['x_1', 'y_1', 'z_1']].values

In [42]:
ai0 = test_df['atom_index_0']
ai1 = test_df['atom_index_1']
type_ = pd.get_dummies(test_df['type'])

euclid_distance = np.linalg.norm(test_p_0 - test_p_1, axis=1)
distance = pd.DataFrame(euclid_distance, columns=['distance'])
larger_atom = pd.DataFrame(ai0 > ai1, columns=['larger_atom'])
index_diff = pd.DataFrame(abs(ai0 - ai1), columns=['index_diff'])

x_0 = test_0['x_0']
y_0 = test_0['y_0']
z_0 = test_0['z_0']

x_1 = test_1['x_1']
y_1 = test_1['y_1']
z_1 = test_1['z_1']

x_dist = pd.DataFrame(abs(x_0 - x_1), columns=['x_dist'])
y_dist = pd.DataFrame(abs(y_0 - y_1), columns=['y_dist'])
z_dist = pd.DataFrame(abs(z_0 - z_1), columns=['z_dist'])



In [43]:
final_test = pd.concat([test_df['id'],test_df['molecule_name'],test_df['type'],type_, ai0, ai1, larger_atom,index_diff,distance,x_0,y_0,z_0,x_1,y_1,z_1], axis = 1) 

In [44]:
add_features(final_test)

In [45]:
final_test = final_test.drop(['id','molecule_name','type'], axis = 1) 


In [46]:
predictions = model_rf.predict(final_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    7.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    9.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:   10.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:   12.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:   14.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   15.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   17.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:   19.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [47]:
output = pd.DataFrame({'id': test_df.id,
                       'scalar_coupling_constant': predictions})
output.to_csv('submission.csv', index=False)